# RAG Pipeline using LlamaIndex

## Pre-requisites

In [ ]:
%pip install llama-index llama-index-embeddings-huggingface llama-index-vector-stores-chroma -q -U

In [ ]:
# load openai api key
import os

from dotenv import load_dotenv
load_dotenv()

if not 'OPENAI_API_KEY' in os.environ:
    raise ValueError('OPENAI_API_KEY is not set')

## Dataset Loader

In [ ]:
# Import the sample dataset from the library
from validmind.datasets.llm.rag import rfp

raw_df = rfp.load_data()
train_df, test_df = rfp.preprocess(raw_df)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## Embedding Model Selection

First, we test both the `sentence-transformers` and `openai` embedding models using their native interfaces. 

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings = embed_model.get_text_embedding("Hello World!")

print(len(embeddings))
print(embeddings[:5])

In [ ]:
from openai import OpenAI
client = OpenAI()

embeddings = client.embeddings.create(
    input="Hello World!", 
    model="text-embedding-3-small"
).data[0].embedding

print(len(embeddings))
print(embeddings[:5])

Create `validmind` embedding models.

In [ ]:
from validmind.models import FunctionModel

def embed(input):
    """Returns a text embedding for the given text"""
    
    embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")
    return embed_model.get_text_embedding(input["question"])


vm_embedder_st = FunctionModel(input_id="embedding_model_st", predict_fn=embed)

In [ ]:
def embed(question):
    """Returns a text embedding for the given text"""

    return (
        client.embeddings.create(
            input=question,
            model="text-embedding-3-small",
        )
        .data[0]
        .embedding
    )


vm_embedder_openai = FunctionModel(
    input_id="embedding_model_openai", 
    predict_fn=embed
)

Create `validmind` datasets.

In [ ]:
import validmind as vm

vm_train_ds = vm.init_dataset(train_df, text_column="question", __log=False)
vm_test_ds = vm.init_dataset(test_df, text_column="question", __log=False)

Generate embeddings from the text in the `question` column.

In [ ]:
vm_test_ds.assign_predictions(vm_embedder_st)
vm_test_ds.assign_predictions(vm_embedder_openai)

Run an embedding test for both models to ensure that the embedding models function properly.

In [ ]:
from validmind.tests import run_test

run = True
if run:
    result = run_test(
        "validmind.model_validation.embeddings.StabilityAnalysisRandomNoise:HFEmbeddingModel",
        inputs={"model": vm_embedder_st, "dataset": vm_test_ds},
        params={"probability": 0.3},
    )

In [ ]:
run = True
if run:
    result = run_test(
        "validmind.model_validation.embeddings.StabilityAnalysisRandomNoise:OpenAIEmbeddingModel",
        inputs={"model": vm_embedder_openai, "dataset": vm_test_ds},
        params={"probability": 0.3},
    )

## Setup Vector Store

#### Generate embeddings for the questions

In [ ]:
train_df["embedding"] = vm_embedder_st.predict(train_df)
train_df.head()

#### Insert embeddings and questions into Vector DB

In [ ]:
import chromadb
import uuid
chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(name="rfp_rag_collection")

# Initialize lists to store data for batch addition
all_embeddings = []
all_metadatas = []
all_documents = []
all_ids = []

# Loop through the DataFrame rows
for index, row in train_df.iterrows():

    all_embeddings.append(row["embedding"])
    all_metadatas.append({
        'ground_truth': row['ground_truth'],
        'hnsw:space': 'cosine'
    })
    all_documents.append(row['question'])
    all_ids.append(str(uuid.uuid4()))

# Add all data to the collection in a single operation
collection.add(
    ids=all_ids, 
    documents=all_documents,
    embeddings=all_embeddings,
    metadatas=all_metadatas,
)

We test the retriever by directly querying using the pre-computed embedding corresponding to the question. We expect the vector store to return the top k most similar questions, along with the metadata associated with each of these questions.

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.vector_stores.types import VectorStoreQuery

chroma_vector_store = ChromaVectorStore(chroma_collection=collection)
query = VectorStoreQuery(query_embedding=list(vm_test_ds.y_pred(vm_embedder_st)[0]), similarity_top_k=10)
result = chroma_vector_store.query(query)

In [ ]:
for node, similarity, id_ in zip(result.nodes, result.similarities, result.ids):
    print("Node ID:", id_)
    print("Question:", node.text)
    print("Answer:", node.metadata['ground_truth'])
    print("Similarity:", similarity)
    print()

## Setup Retrieval Model

In [ ]:
def retrieve(input):

    contexts = []
    
    query = VectorStoreQuery(query_embedding=input["embedding_model_st"], similarity_top_k=10)

    result = chroma_vector_store.query(query)

    for node, similarity, id_ in zip(result.nodes, result.similarities, result.ids):

        context = f"Node ID: {id_}\n"
        context = f"Question: {node.text}\n"
        context += f"Answer: {node.metadata['ground_truth']}\n"
        context += f"Similarity: {similarity}\n"

        contexts.append(context)

    return contexts

vm_retriever = FunctionModel(input_id="retrieval_model", predict_fn=retrieve)

In [ ]:
from validmind.models import PipelineModel

embed_retrieve_pipeline = PipelineModel(vm_embedder_st | vm_retriever, input_id="embed_retrieve_pipeline")

In [ ]:
vm_test_ds.assign_predictions(embed_retrieve_pipeline)
vm_test_ds.df.head()

## Setup Generation Model

In [ ]:
from llama_index.core import Prompt

template = """
Answer the question based only on the following context. 
If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""
prompt = Prompt(template)

In [ ]:
formatted_prompt = prompt.format(
    context=vm_test_ds.df.iloc[0]['embed_retrieve_pipeline_prediction'][0], 
    question=vm_test_ds.df.iloc[0]['question'],
)
print(formatted_prompt)

In [ ]:
from openai import OpenAI


client = OpenAI()

def generate(input):

    formatted_prompt = prompt.format(
        context=input[vm_retriever.input_id], 
        question=input["question"],
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": formatted_prompt},
        ],
    )

    return response.choices[0].message.content

vm_generator = FunctionModel(input_id="generation_model", predict_fn=generate)

## Setup a ValidMind RAG Model

In [ ]:
vm_rag_model = PipelineModel(vm_embedder_st | vm_retriever | vm_generator, input_id="rag_model")

In [ ]:
vm_test_ds.assign_predictions(vm_rag_model)
vm_test_ds.df.head()

In [ ]:
# vm_ragas_ds = vm.init_dataset(result_df, __log=False)

In [ ]:
import plotly.express as px

def plot_distribution(scores):
    # plot distribution of scores (0-1) from ragas metric
    # scores is a list of floats
    fig = px.histogram(x=scores, nbins=10)
    fig.show()

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# result = run_test(
#     "validmind.model_validation.ragas.AnswerSimilarity",
#     inputs={"dataset": vm_ragas_ds},
#     show=False,
# )
# plot_distribution(result.metric.summary.results[0].data)